In [ ]:
import numpy as np
import open3d as o3d

In [ ]:
pcd = o3d.io.read_point_cloud("./Points/clean points/dwnsample_pose_1.ply")
mesh = o3d.io.read_point_cloud("./Mesh/results/jean normal/gauss_pose_1.ply")

In [ ]:
o3d.visualization.draw_geometries([mesh])

In [ ]:
points = np.asarray(pcd.points)
vertices = np.asarray(mesh.vertices)

In [ ]:
for i in vertices:
    if np.array_equal(i,points[0]):
        print('exist')